In [6]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import seaborn as sns



from mpl_toolkits.mplot3d import Axes3D
from sklearn.model_selection import train_test_split, learning_curve
from sklearn.metrics import average_precision_score
from xgboost.sklearn import XGBClassifier
from xgboost import plot_importance, to_graphviz

In [7]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [8]:
#Import data and correct spelling of original column headers for consistency
df = pd.read_csv('data/dataset.csv')
df = df.rename(columns={'oldbalanceOrg':'oldBalanceOrig', 'newbalanceOrig':'newBalanceOrig', \
                        'oldbalanceDest':'oldBalanceDest', 'newbalanceDest':'newBalanceDest'})
print(df.head())

   step      type    amount     nameOrig  oldBalanceOrig  newBalanceOrig  \
0     1   PAYMENT   9839.64  C1231006815        170136.0       160296.36   
1     1   PAYMENT   1864.28  C1666544295         21249.0        19384.72   
2     1  TRANSFER    181.00  C1305486145           181.0            0.00   
3     1  CASH_OUT    181.00   C840083671           181.0            0.00   
4     1   PAYMENT  11668.14  C2048537720         41554.0        29885.86   

      nameDest  oldBalanceDest  newBalanceDest  isFraud  isFlaggedFraud  
0  M1979787155             0.0             0.0        0               0  
1  M2044282225             0.0             0.0        0               0  
2   C553264065             0.0             0.0        1               0  
3    C38997010         21182.0             0.0        1               0  
4  M1230701703             0.0             0.0        0               0  


In [9]:
#feature discription
df.describe()

,step,amount,oldBalanceOrig,newBalanceOrig,oldBalanceDest,newBalanceDest,isFraud,isFlaggedFraud
count,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06
mean,2.433972e+02,1.798619e+05,8.338831e+05,8.551137e+05,1.100702e+06,1.224996e+06,1.290820e-03,2.514687e-06
std,1.423320e+02,6.038582e+05,2.888243e+06,2.924049e+06,3.399180e+06,3.674129e+06,3.590480e-02,1.585775e-03
min,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.560000e+02,1.338957e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,2.390000e+02,7.487194e+04,1.420800e+04,0.000000e+00,1.327057e+05,2.146614e+05,0.000000e+00,0.000000e+00
75%,3.350000e+02,2.087215e+05,1.073152e+05,1.442584e+05,9.430367e+05,1.111909e+06,0.000000e+00,0.000000e+00
max,7.430000e+02,9.244552e+07,5.958504e+07,4.958504e+07,3.560159e+08,3.561793e+08,1.000000e+00,1.000000e+00


In [15]:
#Test if there any missing values in DataFrame, this does not rule out proxies by a numerical value like 0.
df.isnull().values.any()

False

In [ ]:
#How many transcation are fraudulent?

print ('\n The number of total transaction = {}'.\
       format(len(df)))

print ('\n The number of fraudulent transaction = {} %'.\
       format(len(df.loc[(df.isFraud == 0)]))) 

print ('\n The number of non fraudulent transaction = {} %'.\
       format(len(df.loc[(df.isFraud == 1)]))) 

df['isFraud'].value_counts().plot(kind='bar')

#Need to deal with Umbalanced data 

In [21]:
#Which types of transactions are fraudulent?

print('\n The types of fraudulent transactions are {}'.format(list(df.loc[df.isFraud == 1].type.drop_duplicates().values)))

dfFraudTransfer = df.loc[(df.isFraud == 1) & (df.type == 'TRANSFER')]
dfFraudCashout = df.loc[(df.isFraud == 1) & (df.type == 'CASH_OUT')]

print ('\n The number of fraudulent TRANSFERs = {}'.\
       format(len(dfFraudTransfer))) 

print ('\n The number of fraudulent CASH_OUTs = {}'.\
       format(len(dfFraudCashout))) 




 The types of fraudulent transactions are ['TRANSFER', 'CASH_OUT']

 The number of fraudulent TRANSFERs = 4097

 The number of fraudulent CASH_OUTs = 4116


In [13]:
#. What determines whether the feature isFlaggedFraud gets set or not

print ('\n The number of isFlaggedFraud TRANSFERs = {}'.\
       format(len(df.loc[(df.isFlaggedFraud == 1)]))) 

print('\nThe type of transactions in which isFlaggedFraud is set: {}'\
      .format(list(df.loc[df.isFlaggedFraud == 1].type.drop_duplicates()))) 
                                                          
dfTransfer = df.loc[df.type == 'TRANSFER']
dfFlagged = df.loc[df.isFlaggedFraud == 1]
dfNotFlagged = df.loc[df.isFlaggedFraud == 0]

print('\nMin amount transacted when isFlaggedFraud is set= {}'\
    .format(dfFlagged.amount.min())) 

print('\nMax amount transacted in a TRANSFER where isFlaggedFraud is not set=\
 {}'.format(dfTransfer.loc[dfTransfer.isFlaggedFraud == 0].amount.max()))

# a transaction with 'TRANSFER' an 'amount' greater than 200,000 is not enougth to set isFlaggedFraud to 1


 The number of isFlaggedFraud TRANSFERs = 16

The type of transactions in which isFlaggedFraud is set: ['TRANSFER']

Min amount transacted when isFlaggedFraud is set= 353874.22

Max amount transacted in a TRANSFER where isFlaggedFraud is not set= 92445516.64


In [31]:
#Can oldBalanceDest and newBalanceDest determine isFlaggedFraud being set to 1? no
print('\nThe number of TRANSFERs where isFlaggedFraud = 0, yet oldBalanceDest = 0 and\
 newBalanceDest = 0: {}'.\
format(len(dfTransfer.loc[(dfTransfer.isFlaggedFraud == 0) & \
(dfTransfer.oldBalanceDest == 0) & (dfTransfer.newBalanceDest == 0)]))) # 4158





The number of TRANSFERs where isFlaggedFraud = 0, yet oldBalanceDest = 0 and newBalanceDest = 0: 4158

The number of TRANSFERs where isFlaggedFraud = 0, and oldBalanceDest = newBalanceDest:  16 


In [35]:
#isFlaggedFraud being set cannot be thresholded on oldBalanceOrig ?

print('\nMin, Max of oldBalanceOrig for isFlaggedFraud = 1 TRANSFERs: {}'.\
format([round(dfFlagged.oldBalanceOrig.min()), round(dfFlagged.oldBalanceOrig.max())]))

print('\nMin, Max of oldBalanceOrig for isFlaggedFraud = 0 TRANSFERs where \
oldBalanceOrig = \
newBalanceOrig: {}'.format(\
[dfTransfer.loc[(dfTransfer.isFlaggedFraud == 0) & (dfTransfer.oldBalanceOrig \
== dfTransfer.newBalanceOrig)].oldBalanceOrig.min(), \
round(dfTransfer.loc[(dfTransfer.isFlaggedFraud == 0) & (dfTransfer.oldBalanceOrig \
               == dfTransfer.newBalanceOrig)].oldBalanceOrig.max())]))

#Note that we do not need to consider newBalanceOrig since it is updated only after the transaction,
#whereas isFlaggedFraud would be set before the transaction takes place


Min, Max of oldBalanceOrig for isFlaggedFraud = 1 TRANSFERs: [353874.0, 19585040.0]

Min, Max of oldBalanceOrig for isFlaggedFraud = 0 TRANSFERs where oldBalanceOrig = newBalanceOrig: [0.0, 575668.0]


In [37]:
#Can isFlaggedFraud be set based on seeing a customer transacting more than once? 

print('\nHave originators of transactions flagged as fraud transacted more than once? {}'\
.format((dfFlagged.nameOrig.isin(pd.concat([dfNotFlagged.nameOrig, \
                                    dfNotFlagged.nameDest]))).any())) # False

print('\nHave destinations for transactions flagged as fraud initiated other transactions? \
{}'.format((dfFlagged.nameDest.isin(dfNotFlagged.nameOrig)).any())) # False

# Since only 2 destination accounts of 16 that have 'isFlaggedFraud' set have been
# destination accounts more than once,
# clearly 'isFlaggedFraud' being set is independent of whether a 
# destination account has been used before or not

print('\nHow many destination accounts of transactions flagged as fraud have been \
destination accounts more than once?: {}'\
.format(sum(dfFlagged.nameDest.isin(dfNotFlagged.nameDest)))) 



Have originators of transactions flagged as fraud transacted more than once? False

Have destinations for transactions flagged as fraud initiated other transactions? False

How many destination accounts of transactions flagged as fraud have been destination accounts more than once?: 2


In [53]:
#Is the transaction stoped when isFlaggedFraud being set 

print('\n The number of stoped transaction when isFlaggedFraud set to one:  {} \n'.\
format(len(dfTransfer.loc[(dfTransfer.isFlaggedFraud == 1) & \
(dfTransfer.oldBalanceOrig == dfTransfer.newBalanceOrig )])))

#print(df.loc[df.isFlaggedFraud == 1])

#isFlaggedFraud for detecting fraud without interfering so much with good transactions
#when the record has the isFlaggedFraud = 1, this means that the transaction was detected and stopped from being processed.

print('\n The number of rows when isFraud is always set when isFlaggedFraud is set :  {} \n'.\
format(len(dfTransfer.loc[(dfTransfer.isFlaggedFraud == 1) & (dfTransfer.isFraud == 0 )]))) 


#Although isFraud is always set when isFlaggedFraud is set,
#since isFlaggedFraud is set just 16 times in a seemingly meaningless way, 
#we can treat this feature as insignificant and discard it in the dataset without loosing information.
#so it will be good to remove this column IsFraud


 The number of stoped transaction when isFlaggedFraud set to one:  16 


 The number of rows when isFraud is always set when isFlaggedFraud is set :  0 



In [43]:
#Are expected merchant accounts accordingly labelled?

print('\nAre there any merchants among originator accounts for CASH_IN \
transactions? {}'.format(\
(df.loc[df.type == 'CASH_IN'].nameOrig.str.contains('M')).any())) # False

print('\nAre there any merchants among destination accounts for CASH_OUT \
transactions? {}'.format(\
(df.loc[df.type == 'CASH_OUT'].nameDest.str.contains('M')).any())) # False

print('\nAre there merchants among any originator accounts? {}'.format(\
      df.nameOrig.str.contains('M').any())) # False

print('\nAre there any transactions having merchants among destination accounts\
 other than the PAYMENT type? {}'.format(\
(df.loc[df.nameDest.str.contains('M')].type != 'PAYMENT').any())) # False

#Among the account labels nameOrig and nameDest, for all transactions, 
#the merchant prefix of 'M' occurs in an unexpected way.


Are there any merchants among originator accounts for CASH_IN transactions? False

Are there any merchants among destination accounts for CASH_OUT transactions? False

Are there merchants among any originator accounts? False

Are there any transactions having merchants among destination accounts other than the PAYMENT type? False


In [47]:
#Are there account labels common to fraudulent TRANSFERs and CASH_OUTs?

print('\nWithin fraudulent transactions, are there destinations for TRANSFERS \
that are also originators for CASH_OUTs? {}'.format(\
(dfFraudTransfer.nameDest.isin(dfFraudCashout.nameOrig)).any())) # False
dfNotFraud = df.loc[df.isFraud == 0]

print('\nFraudulent TRANSFERs whose destination accounts are originators of \
genuine CASH_OUTs: \n\n{}'.format(dfFraudTransfer.loc[dfFraudTransfer.nameDest.\
isin(dfNotFraud.loc[dfNotFraud.type == 'CASH_OUT'].nameOrig.drop_duplicates())]))

print('\nFraudulent TRANSFER to C423543548 occured at step = 486 whereas \
genuine CASH_OUT from this account occured earlier at step = {}'.format(\
dfNotFraud.loc[(dfNotFraud.type == 'CASH_OUT') & (dfNotFraud.nameOrig == \
                           'C423543548')].step.values)) # 185

#Noting from section 2.3 above that the nameOrig and nameDest features neither encode 
#merchant accounts in the expected way, below, we drop these features from the data since they are meaningless.


Within fraudulent transactions, are there destinations for TRANSFERS that are also originators for CASH_OUTs? False

Fraudulent TRANSFERs whose destination accounts are originators of genuine CASH_OUTs: 

         step      type      amount     nameOrig  oldBalanceOrig  \
1030443    65  TRANSFER  1282971.57  C1175896731      1282971.57   
6039814   486  TRANSFER   214793.32  C2140495649       214793.32   
6362556   738  TRANSFER   814689.88  C2029041842       814689.88   

         newBalanceOrig     nameDest  oldBalanceDest  newBalanceDest  isFraud  \
1030443             0.0  C1714931087             0.0             0.0        1   
6039814             0.0   C423543548             0.0             0.0        1   
6362556             0.0  C1023330867             0.0             0.0        1   

         isFlaggedFraud  
1030443               0  
6039814               0  
6362556               0  

Fraudulent TRANSFER to C423543548 occured at step = 486 whereas genuine CASH_OUT from this 

In [64]:
#Missing data in balances ?
#C1305486145 transfered 181.0 to C553264065 and the balance of C1305486145 decreased by181.0. But why does the balance of the destination C553264065 remain zero?

df.query('step == 1 and (nameOrig == "C553264065" or nameDest == "C553264065")')

print('\nThe number of transaction where BalanceDest doesn t change even IsflaggedFraud not set: {}'.\
format(len(dfTransfer.loc[(dfTransfer.isFlaggedFraud == 0) & \
(dfTransfer.oldBalanceDest == dfTransfer.newBalanceDest)])))

#we have data for the account at origin but not for the account at the destination - 
#perhaps it isn't an account that is registered with the company => missing data



The number of transaction where BalanceDest doesn t change even IsflaggedFraud not set: 4158
